# Hello Friends 👋

Assuming this as a **Multiclass classification** task, I'm trying-out end-to-end classification (*SparseCategoricalCrossentropy Loss*) (linking KAGGLE-data cloud bucket). 


Notebook is for-
* getting started faster
* beginners who want to try out TPU training

Special thanks to-
* https://www.kaggle.com/ks2019/happywhale-arcface-baseline-tpu
* https://www.kaggle.com/docs/tpu
* https://www.kaggle.com/product-feedback/129828

In [ ]:
import numpy as np
import pandas as pd
import os, sys, cv2, math
from kaggle_datasets import KaggleDatasets
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, LearningRateScheduler

In [ ]:
# data.csv excluded id_freq>150
df= pd.read_csv('../input/dataframe-startnotebook/data.csv')
Encoder=LabelEncoder()
df['id_label']=Encoder.fit_transform(df.individual_id)
np.save('classes.npy', Encoder.classes_)
# enc.classes_ = np.load('classes.npy', allow_pickle=True)
# enc.inverse_transform([y1, y2])
df.head()

In [ ]:
n_classes= df.id_label.max()+1
img_size = 600
n_epochs = 40
lr= 0.0001
val_split= 0.2
seed= 2001
batch_size=16
n_classes

## TPU Input Pipeline
Usefull links
* https://www.tensorflow.org/guide/tpu
* https://www.tensorflow.org/guide/data_performance

In [ ]:
def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    
    return strategy

In [ ]:
def readImg(with_labels=True, target_size=(512, 512)):
    def readOnly(path):
        file_bytes = tf.io.read_file(path)
        img = tf.image.decode_jpeg(file_bytes, channels=3)
        img= tf.cast(img, tf.float32)/255.0
        return tf.image.resize(img, target_size)
    def readWithLabels(path, label):
        return readOnly(path), label
    return readWithLabels if with_labels else readOnly

def build_augmenter(with_labels=True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        #img = tf.image.random_flip_up_down(img)
        img = tf.image.random_saturation(img, 0.8, 1.2)
        img = tf.image.random_brightness(img, 0.1)
        img = tf.image.random_contrast(img, 0.8, 1.2)
        return img
    def augment_with_labels(img, label):
        return augment(img), label
    return augment_with_labels if with_labels else augment

def build_dataset(paths, labels=None, bsize=20,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=2001,
                  cache_dir="", cache=True):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
        
    if decode_fn is None:
        decode_fn = readImg(labels is not None)
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
        
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO) # overlaps data preprocessing and model execution while training
    return dset

In [ ]:
DATASET_NAME = "happy-whale-and-dolphin"
strategy = auto_select_accelerator()
batch_size = strategy.num_replicas_in_sync * batch_size
print('batch size', batch_size)

In [ ]:
GCS_DS_PATH = KaggleDatasets().get_gcs_path(DATASET_NAME)
paths = GCS_DS_PATH + "/train_images/" + df['image']
GCS_DS_PATH

In [ ]:
# Train test split
(train_paths, valid_paths, 
  train_labels, valid_labels) = train_test_split(paths, df.id_label.values.reshape(-1,1).astype('float32'),
                                                 test_size=val_split, random_state=seed)

print(train_paths.shape, valid_paths.shape)

In [ ]:
decoder = readImg(with_labels=True, target_size=(img_size, img_size))

# Build the tensorflow datasets
dtrain = build_dataset(
    train_paths, train_labels, bsize=batch_size, decode_fn=decoder)

dvalid = build_dataset(
    valid_paths, valid_labels, bsize=batch_size, 
    repeat=False, shuffle=False, augment=False, decode_fn=decoder)

In [ ]:
data, _ = dtrain.take(2)
images = data[0].numpy()

In [ ]:
fig, axes = plt.subplots(3, 4, figsize=(20,10))
axes = axes.flatten()
for img, ax in zip(images, axes):
    ax.imshow(img)
    ax.axis('off')
plt.tight_layout()
plt.show()

## CNN Model

In [ ]:
# https://www.kaggle.com/ks2019/happywhale-arcface-baseline-tpu

class ArcMarginProduct(tf.keras.layers.Layer):
    '''
    Implements large margin arc distance.

    Reference:
        https://arxiv.org/pdf/1801.07698.pdf
        https://github.com/lyakaap/Landmark2019-1st-and-3rd-Place-Solution/
            blob/master/src/modeling/metric_learning.py
    '''
    def __init__(self, n_classes, s=30, m=0.50, easy_margin=False,
                 ls_eps=0.0, **kwargs):

        super(ArcMarginProduct, self).__init__(**kwargs)

        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.ls_eps = ls_eps
        self.easy_margin = easy_margin
        self.cos_m = tf.math.cos(m)
        self.sin_m = tf.math.sin(m)
        self.th = tf.math.cos(math.pi - m)
        self.mm = tf.math.sin(math.pi - m) * m

    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'n_classes': self.n_classes,
            's': self.s,
            'm': self.m,
            'ls_eps': self.ls_eps,
            'easy_margin': self.easy_margin,
        })
        return config

    def build(self, input_shape):
        super(ArcMarginProduct, self).build(input_shape[0])

        self.W = self.add_weight(
            name='W',
            shape=(int(input_shape[0][-1]), self.n_classes),
            initializer='glorot_uniform',
            dtype='float32',
            trainable=True,
            regularizer=None)

    def call(self, inputs):
        X, y = inputs
        y = tf.cast(y, dtype=tf.int32)
        cosine = tf.matmul(
            tf.math.l2_normalize(X, axis=1),
            tf.math.l2_normalize(self.W, axis=0)
        )
        sine = tf.math.sqrt(1.0 - tf.math.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = tf.where(cosine > 0, phi, cosine)
        else:
            phi = tf.where(cosine > self.th, phi, cosine - self.mm)
        one_hot = tf.cast(
            tf.one_hot(y, depth=self.n_classes),
            dtype=cosine.dtype
        )
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.n_classes

        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        return output

In [ ]:
def build_model():
    inp = layers.Input(shape = (img_size, img_size, 3))
    base=tf.keras.applications.EfficientNetB6(input_tensor= inp, include_top=False,
                                   classes= n_classes, weights='imagenet')
    
    x= base(inp)
    x= layers.GlobalAveragePooling2D()(layers.Dropout(0.15)(x))
    x= layers.Dropout(0.2)(x)
    x= layers.Dense(n_classes, 'softmax')(x)
    return tf.keras.Model(inp, x)

In [ ]:
def step_decay(epoch):
    initial_lrate = 0.0001
    drop = 0.5
    epochs_drop = 5.0
    lrate = initial_lrate* math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

LR_START = 1e-5
LR_MAX = 0.0001
LR_RAMPUP_EPOCHS = 2
LR_SUSTAIN_EPOCHS = 1
LR_STEP_DECAY = 0.7

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = LR_MAX * LR_STEP_DECAY**((epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS)//2)
    return lr

lrate = LearningRateScheduler(lrfn)

In [ ]:
with strategy.scope():
    model= build_model()
    loss= tf.keras.losses.SparseCategoricalCrossentropy()
    model.compile(Adam(lr=lr),loss=loss)
model.summary()

In [ ]:
name= 'EfficientNetB5v1.h5'
ckp = ModelCheckpoint(name,monitor = 'val_loss',
                      verbose = 1, save_best_only = True, mode = 'min')
        
es = EarlyStopping(monitor = 'val_loss', min_delta = 1e-4, patience = 5, mode = 'min', 
                    restore_best_weights = True, verbose = 1)

In [ ]:
steps_per_epoch = ((train_paths.shape[0] // batch_size)//100)*100 - 50
steps_per_epoch

In [ ]:
history = model.fit(dtrain,                      
                    validation_data=dvalid,                                       
                    epochs=n_epochs,
                    callbacks=[es,ckp,lrate],
                    steps_per_epoch=steps_per_epoch,
                    verbose=1)

In [ ]:
plt.figure(figsize = (12, 6))
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.plot( history.history["loss"], label = "Training Loss", marker='o')
plt.plot( history.history["val_loss"], label = "Validation Loss", marker='+')
plt.grid(True)
plt.legend()
plt.show()